# Some Project Euler Solutions in Python 3

December 18, 2019  
Edwin Watkeys

In [1]:
import numpy, itertools, functools, array

#### 1. Multiples of 3 and 5

In [2]:
numpy.sum([i for i in range(1, 1000) if (i % 3 == 0) or (i % 5== 0)])

233168

#### 2. Even Fibonacci numbers

In [3]:
def fibs():
    a = 0
    b = 1
    while True:
        yield a
        a,b = b, a+b

fib_limit = 4000000
my_fibs = itertools.takewhile(lambda x: x <= fib_limit, fibs())
numpy.sum([x for x in my_fibs if x % 2 == 0])

4613732

#### 3. Largest prime factor

In [4]:
def primes():
    """Generate primes."""
    ps = []
    n = 2
    while True:
        our_ps = itertools.takewhile(lambda x: x <= numpy.sqrt(n), ps)
        if len([x for x in our_ps if n % x == 0]) == 0:
            yield n
            ps.append(n)
        n = n + 1

class BitArray:
    """A simple bit array."""
    storage_type = 'L'
    item_bits = 32
    allbits_mask = 0xffffffff
    
    def __init__(self, size, default_value = 0x0):
        l = int(numpy.ceil(size/BitArray.item_bits))
        self.storage = array.array(BitArray.storage_type, [default_value for _ in range(l)])
    
    def set_bit(self, i):
        offset = i // BitArray.item_bits
        mask = 0x01 << i % BitArray.item_bits
        self.storage[offset] = self.storage[offset] | mask
    
    def reset_bit(self, i):
        offset = i // BitArray.item_bits
        mask = 0x01 << i % BitArray.item_bits
        self.storage[offset] = self.storage[offset] & (BitArray.allbits_mask ^ mask)

    def is_bit_set(self, i):
        offset = i // BitArray.item_bits
        mask = 0x01 << i % BitArray.item_bits
        return (self.storage[offset] & mask) != 0
                
def sieve_primes(max_prime):
    """Return a BitArray of primes between zero and max_prime inclusive."""
    a = BitArray(max_prime + 1, 0xffffffff)
    a.reset_bit(0); a.reset_bit(1) # Not primes

    for i in range(2, int(numpy.sqrt(max_prime))+1):
        for j in range(2*i, max_prime+1, i):
            a.reset_bit(j)
    return a
        
def prime_factors(n):
    """Return iterator of prime factors of n from highest to lowest."""
    max_prime = int(numpy.sqrt(n))
    prime_bits = sieve_primes(max_prime)
    our_primes = [i for i in range(2, max_prime+1) if prime_bits.is_bit_set(i)]
    our_primes.reverse()
    while True:
        try:
            factor = next(filter(lambda x: n % x == 0, our_primes))
            yield factor
            n = n / factor
        except StopIteration:
            yield int(n)
            break

<strike>Given that we know that we're never going to be asked for a prime higher than the square root of 600851475143 (roughly 775146.1), I should really just implement Sieve of Erasthones with a bit-array and trade time for space here.</strike>

Using the Sieve of Erasthones, the correct solution is computed in approximately one tenth the time of using the naive `primes()` implementation.

In [5]:
next(prime_factors(600851475143))

6857

#### 4. Largest palindrome product

In [6]:
def is_palindromic(n):
    s = '%s' % (n)
    r1 = range(0, len(s), 1)
    r2 = range(len(s)-1, -1, -1)
    n_tests = (len(r1) // 2) + 1
    matches = [s[r1[x]] == s[r2[x]] for x in range(0, n_tests)]
    return len([x for x in matches if x]) == n_tests

max([x * y for x in range(100, 1000) for y in range(100, 1000) if is_palindromic(x*y)])

906609

#### 5. Smallest multiple

In [7]:
def seq_to_bag(seq):
    bag = {}
    for item in seq:
        if item in bag:
            bag[item] = bag[item] + 1
        else:
            bag[item] = 1
    return bag

def merge_seq_into_bag(seq, bag):
    bag2 = seq_to_bag(seq)
    for x,y in bag2.items():
        if x in bag:
            bag[x] = max(bag[x],y)
        else:
            bag[x] = y
    return bag
            
def lcm_bag(seq):
    bag = {}
    for item in seq:
        merge_seq_into_bag(prime_factors(item), bag)
    return bag

def bag_product(m):
    items = list(m.items())
    return functools.reduce(lambda x,y: x * y[0] ** y[1], items, 1)

In [8]:
bag_product(lcm_bag(range(1,20+1)))

232792560

#### 6. Sum square difference

In [9]:
numpy.abs(sum([x ** 2 for x in range(1, 100+1)]) - sum([x for x in range(1,100+1)]) ** 2)

25164150

#### 7. 10001st prime

In [10]:
def nth(iterable, n):
    return list(itertools.islice(iterable, n, n+1))[0]

nth(primes(), 10001)

104759